In [52]:
import pandas as pd
import numpy as np
from datetime import timedelta
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly

import os
import pickle
from config import BASE_DIR
print(os.getcwd())
os.chdir(BASE_DIR)

import scripts.functions as fn 

C:\Users\lbisa\Mi unidad (leonardo.bisaro@gmail.com)\Cripto\python\trd_app


In [53]:
def loadSymbol(df,symbol):
    #klines_file  = './backtest/klines/1h04/Completo_BTCUSDT_1h04_2021-01-01_2023-07-31.DataFrame'
    #klines_file  = './backtest/klines/1h01/Completo_BTCUSDT_1h01_2021-01-01_2023-07-31.DataFrame'
    #klines_file = './backtest/klines/2d01/Lateral_BTCUSDT_2d01_2023-06-20_2023-10-09.DataFrame'
    #klines_file  = './backtest/klines/2d01/Full_BTCUSDT_2d01_2019-01-01_2024-06-20.DataFrame'
    #klines_file  = './backtest/klines/1h01/Full_BTCUSDT_1h01_2019-01-01_2024-06-20.DataFrame'
    #klines_file  = './backtest/klines/1h01/Completo_BTCUSDT_1h01_2021-01-01_2023-07-31.DataFrame'
    #klines_file  = './backtest/klines/0m15/Completo_BTCUSDT_0m15_2021-01-01_2023-07-31.DataFrame'
    klines_file  = f'./backtest/klines/2d01/Completo_{symbol}_2d01_2021-01-01_2023-07-31.DataFrame'

    with open(klines_file, 'rb') as file:
        df_tmp = pickle.load(file)

    df_tmp['close_pct'] = df_tmp['close'].pct_change() * 100
    
    if df.empty:
        df = pd.DataFrame(columns=['datetime',symbol], index=df_tmp.index)
        df['datetime'] = df_tmp['datetime']
        df[symbol] = df_tmp['close']
    else:
        df[symbol] = df_tmp['close']
    df[f'{symbol}_pct'] = 0.0
    
    
    return df



In [54]:
df = pd.DataFrame()

symbols = ['BTCUSDT','ETHUSDT','BNBUSDT']
symbol_data = {}
symbol_data['BTCUSDT'] = {'decs':9}
symbol_data['ETHUSDT'] = {'decs':6}
symbol_data['BNBUSDT'] = {'decs':5}

row_col = []
row_col.append('violet')
row_col.append('orange')
row_col.append('green')
row_col.append('red')
row_col.append('yellow')

for symbol in symbols:
    df = loadSymbol(df,symbol)
    df[f'{symbol}_qty'] = None


#Filtrar rango de fechas
ini = pd.to_datetime('2021-06-01')
fin = pd.to_datetime('2028-05-01')
df = df[(df['datetime']>=ini)&(df['datetime']<=fin)]
df.reset_index(inplace=True)
df.drop(columns=['index'],inplace=True)

capital_total = 100000.0
capital_limite = capital_total
capital_invertido = 0.0
capital = capital_total * (20/100)



for symbol in symbols:
    qty = round(capital/df.loc[0][symbol],ndigits=symbol_data[symbol]['decs'])
    df.loc[0,f'{symbol}_qty'] = qty
    capital_invertido += capital
    capital_total -= capital

df.loc[0,'capital_total'] = capital_total
df.loc[0,'capital_invertido'] = capital_invertido

#df[0:5]

In [57]:
pre_row = None
pct_chg = 0.05
for index, row in df.iterrows():

    if pre_row is not None:
        #Establece la misma cantidad de unidades que el registro anterior

        full_usd = 0.0
        for symbol in symbols:
            df.loc[index,f'{symbol}_qty'] = pre_row[f'{symbol}_qty']

            if pre_row[f'{symbol}_qty'] * row[f'{symbol}'] > (capital*(1+pct_chg)):
                op_usdt = (pre_row[f'{symbol}_qty'] * row[f'{symbol}'] - capital) * 2
                if op_usdt > 11:
                    sell_qty = op_usdt/row[f'{symbol}']
                    
                    capital_total += op_usdt
                    capital_invertido -= op_usdt
                    df.loc[index,f'{symbol}_qty'] = pre_row[f'{symbol}_qty'] - sell_qty

            elif pre_row[f'{symbol}_qty'] * row[f'{symbol}'] < (capital*(1-pct_chg*2)):
                op_usdt = capital - pre_row[f'{symbol}_qty'] * row[f'{symbol}']
                if op_usdt > 11:
                    buy_qty = op_usdt/row[f'{symbol}']
                    if capital_total-op_usdt >= 0:    
                        capital_total -= op_usdt
                        capital_invertido += op_usdt
                        df.loc[index,f'{symbol}_qty'] = pre_row[f'{symbol}_qty'] + buy_qty




        df.loc[index,'capital_total'] = capital_total
        df.loc[index,'capital_invertido'] = capital_invertido       

    pre_row = df.loc[index]  

df['USDT_h'] = 0
df['USDT_w'] = 0
for symbol in symbols:
    start_qty = df.loc[0][f'{symbol}_qty']
    df[f'{symbol}_usdt_w'] = df[f'{symbol}_qty'] * df[symbol]
    df[f'{symbol}_usdt_h'] = start_qty * df[symbol]
    df['USDT_h'] = df['USDT_h']+ df[f'{symbol}_usdt_h']
    df['USDT_w'] = df['USDT_w']+ df[f'{symbol}_usdt_w']

series = []

fig = make_subplots(rows=1, shared_xaxes=True)
fig.add_trace(
            go.Scatter(
                x=df["datetime"], y=df[f'capital_total'], name=f'Cap.Tot.', mode="lines", 
                line={'width': 0.5},   
                marker=dict(color=row_col[1]),
            ),
            row=1,
            col=1,
        )  
fig.add_trace(
            go.Scatter(
                x=df["datetime"], y=df[f'capital_invertido'], name=f'Cap.Inv.', mode="lines", 
                line={'width': 0.5},   
                marker=dict(color=row_col[2]),
            ),
            row=1,
            col=1,
        )  
fig.add_trace(
            go.Scatter(
                x=df["datetime"], y=df[f'USDT_w'], name=f'USDT_w', mode="lines", 
                line={'width': 0.5},   
                marker=dict(color=row_col[3]),
            ),
            row=1,
            col=1,
        )  
fig.add_trace(
            go.Scatter(
                x=df["datetime"], y=df[f'USDT_w']+df[f'capital_total'], name=f'Cap.Full', mode="lines", 
                line={'width': 0.5},   
                marker=dict(color=row_col[4]),
            ),
            row=1,
            col=1,
        ) 
fig.update_layout(
        font=dict(color="#ffffff", family="Helvetica"),
        paper_bgcolor="rgba(0,0,0,0)",  # Transparent background
        plot_bgcolor="rgba(0,0,0,0)",  # Transparent plot area 
        height=600,
        
        #title="",
        #xaxis_title="",
        #yaxis_title="",

        xaxis=dict(domain=[0, 1]),
        xaxis_rangeslider_visible=False,
 
        modebar_bgcolor="rgba(0,0,0,0)",
        legend=dict(
            orientation = 'h',
            yanchor="bottom",
            y=1.02,
            xanchor="left",
            x=0,
        ),
    )
fig.update_xaxes(showline=True, linewidth=0.5,linecolor='#40444e', gridcolor='#40444e')
fig.update_yaxes(showline=False, linewidth=0.5,zeroline= False, linecolor='#40444e', gridcolor='rgba(0,0,0,0)') 
fig.show()


fig = make_subplots(rows=len(symbols), shared_xaxes=True)
row = 1
color = 0
for symbol in symbols:
    #row += 1
    color += 1
    fig.add_trace(
            go.Scatter(
                x=df["datetime"], y=df[f'{symbol}_qty'], name=f'{symbol} Qty', mode="lines", 
                line={'width': 0.5},   
                marker=dict(color=row_col[color-1]),
            ),
            row=row,
            col=1,
        )   

fig.update_layout(
        font=dict(color="#ffffff", family="Helvetica"),
        paper_bgcolor="rgba(0,0,0,0)",  # Transparent background
        plot_bgcolor="rgba(0,0,0,0)",  # Transparent plot area 
        height=600,
        
        #title="",
        #xaxis_title="",
        #yaxis_title="",

        xaxis=dict(domain=[0, 1]),
        xaxis_rangeslider_visible=False,
 
        modebar_bgcolor="rgba(0,0,0,0)",
        legend=dict(
            orientation = 'h',
            yanchor="bottom",
            y=1.02,
            xanchor="left",
            x=0,
        ),
    )
fig.update_xaxes(showline=True, linewidth=0.5,linecolor='#40444e', gridcolor='#40444e')
fig.update_yaxes(showline=False, linewidth=0.5,zeroline= False, linecolor='#40444e', gridcolor='rgba(0,0,0,0)') 
fig.show()

fig = make_subplots(rows=len(symbols)+1, shared_xaxes=True)
row = 0
for symbol in symbols:
    row += 1
    fig.add_trace(
            go.Scatter(
                x=df["datetime"], y=df[f'{symbol}_usdt_h'], name=f'{symbol} Hold', mode="lines", 
                line={'width': 0.5},   
                marker=dict(color=row_col[row-1]),
            ),
            row=row,
            col=1,
        )   
    fig.add_trace(
            go.Scatter(
                x=df["datetime"], y=df[f'{symbol}_usdt_w'], name=f'{symbol} Wallet', mode="lines", 
                line={'width': 0.5}, 
                marker=dict(color='gray'),  
            ),
            row=row,
            col=1,
        ) 

row += 1
fig.add_trace(
            go.Scatter(
                x=df["datetime"], y=df[f'USDT_h'], name=f'USDT Hold', mode="lines", 
                line={'width': 0.5},   
                marker=dict(color=row_col[row-1]),
            ),
            row=row,
            col=1,
        )   
fig.add_trace(
            go.Scatter(
                x=df["datetime"], y=df[f'USDT_w'], name=f'USDT Wallet', mode="lines", 
                line={'width': 0.5}, 
                marker=dict(color='gray'),  
            ),
            row=row,
            col=1,
        ) 

fig.update_layout(
        font=dict(color="#ffffff", family="Helvetica"),
        paper_bgcolor="rgba(0,0,0,0)",  # Transparent background
        plot_bgcolor="rgba(0,0,0,0)",  # Transparent plot area 
        height=600,
        
        #title="",
        #xaxis_title="",
        #yaxis_title="",

        xaxis=dict(domain=[0, 1]),
        xaxis_rangeslider_visible=False,
 
        modebar_bgcolor="rgba(0,0,0,0)",
        legend=dict(
            orientation = 'h',
            yanchor="bottom",
            y=1.02,
            xanchor="left",
            x=0,
        ),
    )
fig.update_xaxes(showline=True, linewidth=0.5,linecolor='#40444e', gridcolor='#40444e')
fig.update_yaxes(showline=False, linewidth=0.5,zeroline= False, linecolor='#40444e', gridcolor='rgba(0,0,0,0)') 
fig.show()
